# 高大連携プロジェクト
## 「カシマスタジアム行きバスの経由地最適化」

In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd 
import pulp
import csv

In [2]:
#定数
I = list(range(1, 14))   # 経由地候補
S = 0    #　スタート（場所は任意）
G = 14    # ゴール(カシマスタジアム)
U = 3    # バスの台数
M = 300
#入力データ
distance = pd.read_csv('distance.csv', header=None)    # 経由地候補間の距離行列
demand = pd.read_csv('demand.csv', header=None)    # 経由地候補の需要

dem = [0,9, 2, 9, 11, 8, 12, 25, 5, 9, 5, 5, 5, 4, 0]

dis = distance.as_matrix()

/Users/kosakuhajime/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  del sys.path[0]


In [3]:
# 変数
## x : 地点i→地点jを通るか通らないか（０-１変数）
x = {(i, j):pulp.LpVariable('x({},{})'.format(i, j),0,1 ,cat='Integer') for i in list([S])+I for j in I+list([G])}
## t : 経由地候補iを通過する時刻
t = {(i):pulp.LpVariable('t({})'.format(i), lowBound=0, cat='Integer') for i in I}
## y : 地点iから出発する人数
y = {(i):pulp.LpVariable('y({})'.format(i), lowBound=0, cat='Integer') for i in list([S])+I}
## z : 地点iからゴールに行く人数
z = {(i):pulp.LpVariable('z({})'.format(i), lowBound=0, cat='Integer') for i in list([S])+I}

In [4]:
# 問題宣言
prob = pulp.LpProblem("kashima_bus_routes", sense=pulp.LpMaximize)
# 目的関数
prob += pulp.lpSum(z[i] for i in I)

In [5]:
# 制約式

## 各経由地候補から出発・到着するバスの台数は一定
prob += pulp.lpSum(x[S,i] for i in I+list([G])) == U
prob += pulp.lpSum(x[i,G] for i in list([S])+I) == U

## 
for j in I:
    prob += pulp.lpSum(x[i,j] for i in list([S])+I) ==  pulp.lpSum(x[j, k] for k in I+list([G]))
    prob += pulp.lpSum(x[i,j] for i in list([S])+I) <= 1

## 
for i in I:
    prob += x[i,i] == 0

## スタートとゴールはつながらない
prob += x[S,G] <= 0
prob += x[S,G] >= 0

## スタート時の乗客は0人
prob += y[S] == 0

##
for i in list([S])+I:
    for j in I:
        prob += -M*(1-x[i, j])+y[j] <= y[i]+dem[j]

##
for i in I:
    prob += -M*(1-x[i, G]) + z[i] <= y[i]

## 乗車人数は25人以上，45人以下
for i in list([S])+I:
    prob += z[i] <= 45*x[i,G] 
    prob += 25*x[i,G] <=  z[i]

## 
for i in I:
    for j in I:
        prob += -200*(1-x[i,j])+dis[i, j] <= t[i]

## 
for i in I:
    prob += -200*(1-x[i, G])+dis[i, G] <= t[i]

## 自家用車で直接行く場合の1.5倍以内に収める
for i in I:
    prob += t[i] <= 1.5*dis[i, G]

In [ ]:
# ソルバー計算
prob.solve()

In [ ]:
# 結果の出力
## 所要時間
print("　i 　　j : 地点iから地点jへの所要時間")
for i in list([S])+I: 
    for j in I+list([G]):
        if pulp.value(x[i,G]) == 1:
            print(i,"", j, ": minute[", dis[i, j], "]" )

In [ ]:
## バスキャリー            
print("bus carry -/-")
for i in I:
    if x[i, G] == 1:
        print(i, "", pulp.value(z[i]))

In [ ]:
for i in list([S])+I:
    for j in list([S])+I:
        print(dis[i, j], " ")
    print("")